In [6]:
# =============================================================================
# PROJE ADI: CHAOS-LOGISTIC-RNG (Kaos Tabanlı Lojistik Rastgele Sayı Üreteci)
# DERS     : Bilgi Sistemleri ve Güvenliği
# KONU     : Rastgele Sayı Üreteci (RNG) Tasarımı ve İstatistiksel Analizi
# =============================================================================

import time
import math
from collections import Counter

class ChaosLogisticRNG:
    """
    Kaos Teorisindeki 'Lojistik Harita' (Logistic Map) denklemini temel alan
    Sözde Rastgele Sayı Üreteci (PRNG) Sınıfı.

    Temel Denklem: x_{n+1} = r * x_n * (1 - x_n)
    """
    def __init__(self):
        # r parametresi: Kaos bölgesinin derinliklerinde (3.999...) seçilmiştir.
        # Bu değer, sistemin periyodik olmayan kaotik davranış sergilemesini sağlar.
        self.r = 3.999999

        # Tohum (Seed) Belirleme:
        # Sistem zamanının mikrosaniye hassasiyetindeki değeri kullanılarak
        # her çalıştırıldığında farklı bir başlangıç noktası (x0) elde edilir.
        t = time.time()
        self.x = t - int(t)  # Sadece ondalık kısmı al (0.0 - 1.0 arası)

        # Matematiksel kilitlenmeyi önlemek için x 0 veya 1 olmamalıdır.
        if self.x == 0 or self.x == 1:
            self.x = 0.54321

    def generate_bit(self):
        """
        Tek bir rastgele bit (0 veya 1) üretir.
        Yöntem: Lojistik harita çıktısının virgülden sonraki yüksek entropili
        basamakları modülo işlemine tabi tutulur.
        """
        # 1. Adım: Kaotik Denklemi İşlet
        self.x = self.r * self.x * (1 - self.x)

        # 2. Adım: Bit Çıkarımı (Bit Extraction)
        # Ondalık sayıyı büyütüp tamsayıya çeviriyoruz.
        # 10^14 çarpanı, virgülden sonraki kaotik değişimin en yoğun olduğu bölgeyi hedefler.
        scaled_val = int(self.x * (10**14))

        # 3. Adım: Mod 2 ile Bit Elde Etme
        return scaled_val % 2

    def generate_sequence(self, length):
        """Belirtilen uzunlukta bit dizisi döndürür."""
        return [self.generate_bit() for _ in range(length)]

# =============================================================================
# İSTATİSTİKSEL TEST FONKSİYONLARI (NIST SP 800-22 Standartlarına Uygun Mantık)
# =============================================================================

def frequency_test(bits):
    """
    Amaç: Üretilen dizideki 0 ve 1 sayılarının birbirine yakınlığını test eder.
    İdeal durumda oran %50 - %50 olmalıdır.
    """
    total = len(bits)
    zeros = bits.count(0)
    ones = bits.count(1)

    print("\n" + "="*60)
    print("TEST 1: FREKANS (MONOBIT) ANALİZİ")
    print("="*60)
    print(f"[-] Toplam Bit Sayısı : {total}")
    print(f"[-] 0 Sayısı (Zeros)  : {zeros} (Oran: %{zeros/total*100:.4f})")
    print(f"[-] 1 Sayısı (Ones)   : {ones}  (Oran: %{ones/total*100:.4f})")

    diff = abs(zeros - ones)
    print(f"[-] Fark (Delta)      : {diff}")

    return zeros, ones

def chi_square_test(zeros, ones, total):
    """
    Amaç: Ki-Kare (Chi-Square) Testi.
    Gözlenen değerler ile beklenen değerler arasındaki uyumu ölçer.
    H0 Hipotezi: Dizi rastgeledir.
    """
    expected = total / 2  # Beklenen değer

    # Ki-Kare Formülü: sum((Gözlenen - Beklenen)^2 / Beklenen)
    chi_sq = ((zeros - expected)**2 / expected) + ((ones - expected)**2 / expected)

    print("\n" + "="*60)
    print("TEST 2: Kİ-KARE (CHI-SQUARE) UYUMLULUK TESTİ")
    print("="*60)
    print(f"[-] Hesaplanan X² Değeri : {chi_sq:.4f}")

    # 1 Serbestlik derecesi (DoF) ve 0.05 anlamlılık düzeyi (p=0.05) için
    # Kritik Eşik Değer (Critical Value) = 3.841
    CRITICAL_VALUE = 3.841
    print(f"[-] Kritik Eşik Değer    : {CRITICAL_VALUE} (p=0.05)")

    if chi_sq < CRITICAL_VALUE:
        print("[+] SONUÇ: BAŞARILI ✅")
        print("    (X² < 3.841 olduğu için H0 hipotezi kabul edildi.")
        print("    Dağılım istatistiksel olarak rastgele kabul edilebilir.)")
    else:
        print("[-] SONUÇ: BAŞARISIZ ❌")
        print("    (Dağılım rastgelelikten sapma gösteriyor.)")

def runs_test(bits):
    """
    Amaç: Koşu (Runs) Testi (Wald-Wolfowitz).
    Ardışık gelen aynı değerli bitlerin (blokların) sayısını analiz eder.
    Örn: 0011100 -> 3 Koşu (00, 111, 00)
    """
    runs = 1
    for i in range(len(bits)-1):
        if bits[i] != bits[i+1]:
            runs += 1

    n1 = bits.count(0)
    n2 = bits.count(1)
    n = len(bits)

    # İstatistiksel Hesaplamalar
    expected_runs = ((2 * n1 * n2) / n) + 1 # Beklenen koşu sayısı
    variance = (2 * n1 * n2 * (2 * n1 * n2 - n)) / ((n**2) * (n - 1)) # Varyans
    std_dev = math.sqrt(variance) # Standart Sapma

    # Z-Skoru Hesaplama
    z_score = (runs - expected_runs) / std_dev

    print("\n" + "="*60)
    print("TEST 3: KOŞU (RUNS) TESTİ (Bağımsızlık Analizi)")
    print("="*60)
    print(f"[-] Gözlenen Koşu Sayısı : {runs}")
    print(f"[-] Beklenen Koşu Sayısı : {expected_runs:.4f}")
    print(f"[-] Standart Sapma       : {std_dev:.4f}")
    print(f"[-] Z-Skoru (Z-Score)    : {z_score:.4f}")

    # Z-Skoru -1.96 ile +1.96 arasındaysa %95 güven aralığında rastgeledir.
    if -1.96 < z_score < 1.96:
        print("[+] SONUÇ: BAŞARILI ✅")
        print("    (Z-Skoru ±1.96 aralığında. Bitlerin dizilimi bağımsızdır.)")
    else:
        print("[-] SONUÇ: BAŞARISIZ ❌")
        print("    (Bitler arasında kümelenme veya desen tespit edildi.)")

# =============================================================================
# ANA PROGRAM BLOĞU
# =============================================================================

if __name__ == "__main__":
    print(">>> SİSTEM BAŞLATILIYOR: Chaos-Logistic-RNG v1.0")
    print(">>> Yöntem: Kaos Teorisi / Lojistik Harita Dönüşümü\n")

    # 1. Nesne Türetme
    rng = ChaosLogisticRNG()

    # 2. Veri Üretimi
    # İstatistiksel geçerlilik için en az 1000 bit önerilir.
    BIT_AMOUNT = 2000
    print(f">>> {BIT_AMOUNT} adet rastgele bit üretiliyor... Lütfen bekleyiniz.")

    start_time = time.time()
    random_sequence = rng.generate_sequence(BIT_AMOUNT)
    end_time = time.time()

    print(f">>> Üretim Tamamlandı. Süre: {(end_time - start_time):.4f} saniye.\n")

    # 3. Örnek Çıktı Gösterimi (İlk 64 Bit)
    print(f"Örnek Çıktı (İlk 64 Bit):\n{random_sequence[:64]} ...\n")

    # 4. Testlerin Uygulanması
    n_zeros, n_ones = frequency_test(random_sequence)
    chi_square_test(n_zeros, n_ones, BIT_AMOUNT)
    runs_test(random_sequence)

    print("\n" + "="*60)
    print(">>> TÜM ANALİZLER TAMAMLANDI.")
    print("="*60)

>>> SİSTEM BAŞLATILIYOR: Chaos-Logistic-RNG v1.0
>>> Yöntem: Kaos Teorisi / Lojistik Harita Dönüşümü

>>> 2000 adet rastgele bit üretiliyor... Lütfen bekleyiniz.
>>> Üretim Tamamlandı. Süre: 0.0008 saniye.

Örnek Çıktı (İlk 64 Bit):
[1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] ...


TEST 1: FREKANS (MONOBIT) ANALİZİ
[-] Toplam Bit Sayısı : 2000
[-] 0 Sayısı (Zeros)  : 1037 (Oran: %51.8500)
[-] 1 Sayısı (Ones)   : 963  (Oran: %48.1500)
[-] Fark (Delta)      : 74

TEST 2: Kİ-KARE (CHI-SQUARE) UYUMLULUK TESTİ
[-] Hesaplanan X² Değeri : 2.7380
[-] Kritik Eşik Değer    : 3.841 (p=0.05)
[+] SONUÇ: BAŞARILI ✅
    (X² < 3.841 olduğu için H0 hipotezi kabul edildi.
    Dağılım istatistiksel olarak rastgele kabul edilebilir.)

TEST 3: KOŞU (RUNS) TESTİ (Bağımsızlık Analizi)
[-] Gözlenen Koşu Sayısı : 1010
[-] Beklenen Koşu Sayısı : 999.6310
[-] Standa